Este proyecto busca crear un modelo el cual detecte el cancer en la piel mediante datos estructurados (registros medicos de pacientes) y datos no estructurados(imagenes de la piel de los pacientes)

1. *Importacion de librerias *


In [14]:
#Estaremos importando las librerias necesarias para el analisis exploratorio de los datos (EDA) y extraer la informacion
    #tomar en cuenta que mas adelante se importaran librerias para ML
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import requests as rq 
import os 
import json
import time

2. *Importacion de datos no estructurados*

In [ ]:
#Uso de la APIimport requests

# Diagnósticos que nos interesan
tipos_deseados = ["melanoma", "basal cell carcinoma", "squamous cell carcinoma"]

# Configuración de la API
base_url = "https://api.isic-archive.com/api/v2/images"
limit = 100  # cantidad por página
max_pages = 30  # número máximo de páginas a recorrer (ajustable)

nuevos_registros = []

archivo_csv="data_completa"

if os.path.exists(archivo_csv):
    df_existente= pd.read(archivo_csv)
    ids_existentes=set(df_existente["isic_id"])
    print(f"✅ Se cargaron {len(ids_existentes)} registros existentes.")
else:
    df_existente= pd.DataFrame()
    ids_existentes= set()
    print("⚠️ No hay archivo anterior, se creará uno nuevo.")

for page in range(max_pages):
    print(f"🔄 Descargando página {page + 1}/{max_pages}...")
    #Conectar a la api
    url = f"{base_url}?limit={limit}&offset={page * limit}"
    response= rq.get(url)
    #Verificar que todo este correcto, 200= Ok
    if response.status_code != 200:
        print(f"❌ Error en la página {page + 1}: {response.status_code}")
        break
    
    # Convertimos la respuesta a JSON y sacamos los resultados
    data= response.json()["results"]
    #Recorremos cada imagen
    for img in data:
        isic_id= img.get("isic_id")
        if isic_id in ids_existentes:
            continue
        #extraer datos clinicos
        clinical= img.get("metadata", {}).get("clinical", {})
        diagnosis=clinical.get("diagnosis_2","")
        
        #saltar si no tiene diagnosticos
        if diagnosis=="":
            continue
        nuevos_registros.append({"isic_id": isic_id,
            "sexo": clinical.get("sex"),
            "edad": clinical.get("age_approx"),
            "zona_general": clinical.get("anatom_site_general"),
            "diagnostico": diagnosis,
            "resolucion_x": img.get("metadata", {}).get("acquisition", {}).get("pixels_x"),
            "resolucion_y": img.get("metadata", {}).get("acquisition", {}).get("pixels_y"),
            "url_imagen": img.get("files", {}).get("full", {}).get("url")
        })
        
        time.sleep(0.5)
df_nuevos=pd.DataFrame(nuevos_registros)
df_final= pd.concat([df_existentes,df_nuevos],ignore_index=True)

df_final.to_csv(archivo_csv,index=False)
print("\n✅ Proceso completado.")
print(f"📥 Nuevos registros añadidos: {len(df_nuevos)}")
print(f"📦 Total de registros en el archivo: {len(df_final)}")
        

⚠️ No hay archivo anterior, se creará uno nuevo.
🔄 Descargando página 1/30...
🔄 Descargando página 2/30...
🔄 Descargando página 3/30...
🔄 Descargando página 4/30...
🔄 Descargando página 5/30...
🔄 Descargando página 6/30...
🔄 Descargando página 7/30...
🔄 Descargando página 8/30...
🔄 Descargando página 9/30...
🔄 Descargando página 10/30...


In [28]:
df["diagnostico"].unique()

array(['Malignant melanocytic proliferations (Melanoma)'], dtype=object)